In [1]:
import os
import datetime
import random

from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, Band
from bokeh.embed import components

import numpy as np
import math
import scipy.optimize as optim
import pandas as pd

In [2]:
output_notebook()

Loading BokehJS ...

# Introduction to Curve Fitting

Goals for this module¶

* Understand exponential functions
* Understand that SSR is used to fit a curve to a function
* Understand what parameters are
* Understand the null hypothesis for an exponential function and how to determine the p value.
* Understand the rt values for a pandemic

Most of the time, you should use linear regression to find patterns in data. Only in special circumstances should you try to fit your data to a non linear function. 

One such case is modeling covid19. A pandemic grows exponentially. 

## Exponential Functions

An exponential function has an *initial* value, and changes according to a *ratio*. Consider a function that has an initial value of 1, and a ratio of 2. Then the first value, $x_1$ is 1. To get the next value in the series, we multiply the previous value by the ratio. So $x_2$ = 2 * 1 = 2. To get $x_3$, we multiply $x_2$ by 2 to get 4. The first 6 numbers in this series is [1, 2, 4, 8, 16, 32].

The *initial* value and *ratio* are known as parameters.

## Finding the parameters of a function.
Normally, we don't know the parameters. Instead, given a set of numbers, we wish to determine the parameters. 

Consider an experiment in which we test the absorption of a drug over time. As the time increases, the absorption decreases. 

![title](data/exp_fit2.png)

The left hand graph shows the data points. The middle graphs shows a fitted line to the points. The right graph shows a second attempt at a fit. Clearly, Fit 2 is better than Fit 1 because Fit 2 is closer to the points. But how can we mathmatically determine which point is better? 

The fitted line is called y hat, $\hat{y}$. The actual points are called residuals. For each point, we take the difference between the fitted line ($\hat{y}$) and the residual. We square this difference. 


![title](data/ssr1.jpeg)

We do this for each point and add all these squared distances. This is knows as SSR, the *sum of square residuals*. The smaller SSR, the better. In order to find a curve, we minimize this measurement. In math:

$SSR = \sum_{i=1}^n (y_i - \hat{y_i})^2$

In order to solve this equation, we use a function that keeps trying different values and narrows in on a solution. Lets do an example.

### Risk and Alcohol data

The following dataset consists of two columns. BAC is the "blood alcohol level." As the BAC increases, the risk of an accident increases. It does so in an exponential fashion.


In [3]:
DF = pd.read_csv(os.path.join('data', 'alchohol_risk.csv'))
DF.head()

,BAC,risk
0,0.00,1.00
1,0.01,1.03
2,0.03,1.06
3,0.05,1.38
4,0.07,2.09


In [4]:
def resample(l):
    final = []
    for i in range(len(l)):
        final.append(random.choice(l))
    return final

In [5]:
def exp_func(x, initial, ratio):
    return initial * np.power(ratio, x - 1)

We are already acquainted with the resample function. The exp_func takes the list of numbers (x), an initial value, and a ratio. Let's graph the values.

In [6]:
def make_scatter(df, title = None):
    p  = figure(title = title)
    p.circle(x = df['BAC'], y = df['risk'])
    return p
show(make_scatter(DF))

Let's fit a line to these points by finding the parameters. 

In [7]:
popt, pcov = optim.curve_fit(f = exp_func, xdata =DF['BAC'], ydata = DF['risk'])
print('ratio is {r}'.format(r = popt[1]))
print('initial value is {i}'.format(i = popt[0]))

ratio is 57700487606.35652
initial value is 32241472356.689198


In [8]:
# lets' generate a 30 x points
new_x = np.linspace(min(DF['BAC']), max(DF['BAC']), 30)
y_hat = [exp_func(initial = popt[0], ratio = popt[1], x = x) for x in new_x]
p = make_scatter(DF, title = None)
p.line(x = new_x, y = y_hat)
show(p)

## Determine a P value
How good is our fit? To answer this question, we need to state null hypothesis and determine a p value. For an exponential function, the null hypothesis is that the ratio is 1. If the data is just random, on average, the value of each x will sometimes be greater than the previous, and sometimes be smaller. if the ratio is under less than or greater than 1, then we can reject the null hypothesis. 

In [9]:
def get_ratios(x, y, num_iter = 100):
    zip_obj = list(zip(x, y))
    ratios = []
    for i in range(num_iter):
        new_ = resample(zip_obj)
        new_ = sorted(new_, key = lambda x: x[0])
        x_ = [x[0] for x in new_]
        y_ = [x[1] for x in new_]
        popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(x_), ydata = np.array(y_))
        ratios.append(popt[1])
    return ratios

ratios = get_ratios(x = DF['BAC'].tolist(), y = DF['risk'].tolist())
lower = np.percentile(ratios, 2.5)
upper = np.percentile(ratios, 97.5)
print(lower, upper)
lies_inside =  lower < 1 < upper
lies_inside
# reject null hypothesis


/home/henry/Envs2/xianfei_data/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in power
  


23917899040.824837 6404064072673.453


False

In fact, the fit is so good, we can use a 99% confidence interval:

In [10]:
np.percentile(ratios, .5) < 1 < np.percentile(ratios, 99.5)


False

Our p value < .01. We reject the null hypothesis that the ratio is 1

### The p-value for random data
Below is an example of a bad fit. The function *test_random* generates 30 random values for x and y. These random values have no relationship to each other, so we expect the ratio to lie within the confidence interval. 

In [24]:
def test_random():
    # randomly generate some numbers
    x = [random.randrange(100) for x in range(30)]
    y = [random.randrange(100) for x in range(30)]
    # sort them for the graph
    both = list(zip(x,y))
    both = sorted(both, key = lambda x: x[0])
    x = [x[0] for x in both]
    y = [x[1] for x in both]
    # graph them
    p = figure()
    p.circle(x = x, y = y)
    popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(x), ydata = np.array(y))
    y_hat = [exp_func(x = x, ratio = popt[1], initial = popt[0])  for x in x]
    p.line(x = x, y = y_hat, legend_label = 'fitted')
    # the null hypothesis is simply a straight line through the mean of Y
    mean  = np.mean(y)
    p.line(x = x, y = [mean for x in x], color = 'green', legend_label = 'Null')
    print('ratio is {r}'.format(r = popt[1]))
    resamples =  get_ratios(x, y)
    lower = np.percentile(resamples, 2.5)
    upper = np.percentile(resamples, 97.5)
    print(lower, upper)
    lies_inside =  lower < 1 < upper
    print(lies_inside)
    # do not reject null hypothesis

    show(p)
test_random()


ratio is 0.9966069598426586
0.986451258234917 1.0070336102893398
True


After resampling, you will find that the ratio often lies within the confidence interval. (Run the function several times.)

Note that the "Null" or green line is just about as good as fitting a line with an exponential function.

## RT value for Covid-19
A pandemic such as covid-19 grows exponentially. The ratio is known as the rt value. 

For example, imagine that one week there are 2,500 cases in one week. The next week, there are 3,500 cases. The rt is $3,500/2,500 = 1.4$ Since the rt is above 1, the number of infections increases. In fact, 1.4 is very high: in a short amount of time, the pandemic will be out of control. Here is an example of calculating the rt value for Seattle, Washington for the last 14 days. 

In [12]:
# from the previous section. Go to next block
def make_bar(labels, nums, title = None, y_range = None, plot_width = 350, plot_height = 350):
    p = figure(title = title, plot_width = plot_width, plot_height = plot_height,
              y_range = y_range)
    p.vbar(x=labels, top=nums, width=0.9)
    p.xgrid.grid_line_color = None
    return p


In [13]:
#number of cases per day
seattle = [160.0, 158.0, 202.0, 111.0, 124.0, 145.0, 167.0, 192.0, 134.0, 200.0, 129.0, 148.0, 81.0, 149.0]
#note, to get the x values, we simply assign a number, starting at 0 and increasing, for each y
# we do this because this is a time series
seattle_x = range(len(seattle))
popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(seattle_x), ydata = np.array(seattle))
print('rt is {r}'.format(r = popt[1]))
resamples =  get_ratios(seattle_x, seattle)
lower = np.percentile(resamples, 5)
upper = np.percentile(resamples, 95)
print(lower, upper)
lies_inside =  lower < 1 < upper
print(lies_inside)# do not reject 
# let's graph it
p = make_bar(labels = seattle_x, nums = seattle)
p.line(x = seattle_x, 
       y = [exp_func(x =x, initial =popt[0], ratio=popt[1]) for x in seattle_x],
      color = 'red', line_width = 2, legend_label = 'fitted')
p.line(x = seattle_x, y = [np.mean(seattle) for x in seattle_x], color = 'green',
      line_width =2, legend_label = "null hypothesis")

show(p)


rt is 0.9861979922862778
0.964206392311426 1.0073968645374742
True


The rt value for Seattle is less than 1, meaning the pandemic is decreasing. However, the p value is too large, so we cannot reject the null hypothesis that the rt is different than 1. The decrease is not statistical significant. The green line, with no change, is sometimes as good a fit as the red line.


The change may be random. This is one of the challenges of a pandemic, knowing when cases are really increasing, and when to take action, and when the change is not real.  

## Summary
* We fit a curve to data by minimizing SSR (sum of square residuals).
* We resample the samples to generate many fits, and from this derive a p value.
* The null hypothesis for an exponential fit is that the ratio is 1.